Wczytajmy oczyszczone dane

In [14]:
import json
from datetime import datetime

In [15]:
folder = 'data'

with open(folder + '/events.json') as file:
    events_list = json.load(file)

with open(folder + '/products.json') as file:
    products_list = json.load(file)

with open(folder + '/users.json') as file:
    users_list = json.load(file)


Przygotujmy atrybuty do modelu
- miejsce zamieszkania
- cena produktu
- ile razy użytkownik wyświetlił wcześniej dany produkt
- ile produktów danej kategorii obejrzał użytkownik
- ile razy użytkownik kupił produkt z daną zniżką lub niższą
- zniżka


In [16]:
class User:
    def __init__(self, city):
        self.city = city

        self.products_views_counts = dict()

        self.category_views_counts = dict()

        self.bought_with_discount_counts = dict()

        self.bought_category_counts = dict()

users = dict()

for user in users_list:
    users[user['user_id']] = User(user['city'])


class Product:
    def __init__(self, product_id,  price, category):
        self.product_id = product_id
        self.price = price
        self.category = category

products = dict()

for product in products_list:
    products[product['product_id']] = Product(product['product_id'], product['price'], product['category_path'])

class Event:
    def __init__(self, user, datetime, product, bought, discount,session_id):
        self.user = user
        self.datetime = datetime
        self.product = product
        self.bought = bought
        self.discount = discount
        self.session_id = session_id

events = []


for event in events_list:
    bought = False if event['event_type'] == 'VIEW_PRODUCT' else True
    if bought:
        events[-1].bought = True
    else:
        user = users[event['user_id']]
        product = products[event['product_id']]
        datetime = datetime.strptime(event['timestamp'], '%Y-%m-%dT%H:%M:%S')
        events.append(Event(user,datetime, product,bought, event['offered_discount'], event['session_id']))


class Record:
    def __init__(self, city, price, product_views_count, category_view_count,
                 bought_with_lower_count ,bought_category_count,  discount,bought):
        self.city = city
        self.price = price
        self.product_views_count = product_views_count
        self.category_view_count = category_view_count
        self.bought_with_lower_count = bought_with_lower_count
        self.bought_category_count = bought_category_count
        self.discount = discount
        self.bought = bought


records = []

for e in events:

    product_views_count = e.user.products_views_counts.get(e.product.product_id, 0)
    e.user.products_views_counts[e.product.product_id] = product_views_count + 1

    category_view_count = e.user.category_views_counts.get(e.product.category, 0)
    e.user.category_views_counts[e.product.category] = category_view_count + 1

    bought_with_lower_count = 0
    for d in range(0,e.discount + 1, 5):
        bought_with_lower_count += e.user.bought_with_discount_counts.get(d, 0)

    bought_category_count = e.user.bought_category_counts.get(e.product.category, 0)

    if e.bought:
        e.user.bought_with_discount_counts[e.discount] = e.user.bought_with_discount_counts.get(e.discount,0) + 1
        e.user.bought_category_counts[e.product.category] = bought_category_count + 1


    r = Record(e.user.city, e.product.price, product_views_count, category_view_count,bought_with_lower_count,
               bought_category_count, e.discount, e.bought)
    records.append(r)

with open('records.json', 'w') as file:
    json.dump([r.__dict__ for r  in records],file, indent=2)

print("Zapisano rekordy w pliku json")

Zapisano rekordy w pliku json
